In [20]:
import numpy as np
import cv2
from scipy.spatial.distance import euclidean
from collections import defaultdict
import time
import gc


def drawTracks(good_new, good_old, mask, frame):
    # Отрисовка отслеживания перемещения объекта
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        if feature:
            if (euclidean(new, old)) > .4 and a-c > 3:
                pnts[(a,b)] = pnts[(c,d)] + 1 if pnts[(c,d)] != -1 else 1
                dist[(a,b)]['x'] = max(dist[(c,d)]['x'], a-c, key=abs)
                dist[(a,b)]['y'] = max(dist[(c,d)]['y'], b-d, key=abs)
            else:
                pnts[(a,b)] = pnts[(c,d)] 

            pnts[(c,d)] = -1
            dist[(c,d)]['x'], dist[(c,d)]['y'] = 0, 0

            if (c,d) in pnts_g_sc:
                pnts_g_sc.remove((c,d))
                pnts_g_sc.append((a,b))

            if (c,d) in pnts_g:
                pnts_g.remove((c,d))
                pnts_g.append((a,b))
                mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
                frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
                cv2.line(img=frame, pt1=(350, 0), pt2=(350, 1000), color=(255, 0, 0), thickness=5, lineType=8, shift=0)
        else:
            mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv2.circle(frame,(a,b),5,color[i].tolist(), -1)
            cv2.line(img=frame, pt1=(350, 0), pt2=(350, 1000), color=(255, 0, 0), thickness=5, lineType=8, shift=0)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    return k
    

# Включение эффекта
feature = True

# Чтение видеофайла
cap = cv2.VideoCapture('helicopter.mp4') 

# параметры для выделения углов Ши-Томаси
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Параметры для потока Лукаса-Канаде 
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Создание случайных цветов для точек
color = np.random.randint(0,255,(100,3))

# Чтение первого кадра и анализ углов в нём
ret, old_frame = cap.read()
# Преобразование в полутоновой формат
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# Вычисление углов по методу Ши-Томаси
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Создание маскирующего изображения для отрисовки
mask = np.zeros_like(old_frame)

# Получение нового значения
if feature:
    new_g_p = []
    # Цикл по найденным углам
    for p in p0:
        # Сохранение координат найденного угла
        x, y = p[0][0], p[0][1]
        new_g_p.extend([[[x+15, y]], [[x-15, y]], [[x, y-15]], [[x, y+15]]])
        break
    p0 = np.float32(np.append(p0, np.array(new_g_p), axis=0))
    pnts = defaultdict(lambda: -1)
    pnts_g = []
    pnts_g_sc = []

    dist = defaultdict(lambda: {'x':0, 'y':0})

#frame_num - кол кадров, в какой момент проверяем точки
frame_num = 0
minNumGoodPnt = 20
while(1):
    frame_num += 1
    if feature:
        if frame_num == 40:
            new_g_p = []
            max_x, max_y = 0, 0
            for pnt in dist:
                if dist[pnt]['x'] > max_x:
                    max_x = dist[pnt]['x']
                if dist[pnt]['y'] > max_y:
                    max_y = dist[pnt]['y']
            frame_num = 0
            
            # 20 - минимальное кол "хороших" перемещений точек
            for pnt in pnts:
                if pnts[pnt] > minNumGoodPnt and pnts[pnt] > -1:
                     pnts_g.append(pnt)
                elif pnt in pnts_g_sc:
                    pnts_g.remove(pnt)
                    pnts_g_sc.remove(pnt)
                elif pnt in pnts_g:
                    pnts_g.remove(pnt)
                    new_point = np.add(pnt, (max_x, 0))
                    new_g_p.append(tuple(new_point))
                    pnts_g.append(tuple(new_point))
                    pnts_g_sc.append(tuple(new_point))
                pnts[pnt] = 0
            if new_g_p:
                new_g_p = np.array(new_g_p).reshape(-1,1,2)
                p0=np.float32(np.append(p0, new_g_p, axis=0))
            dist = defaultdict(lambda: {'x':0, 'y':0})
            pnts = defaultdict(lambda: -1)
    ret,frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Отслеживание объекта с помощью метода Лукаса-Канаде
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Выбор хороших точек
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    k = drawTracks(good_new, good_old, mask, frame)
    if k == 27:
        break

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()